In [1]:
%matplotlib inline
%load_ext memory_profiler

import os, sys, glob, re, math, pickle
import phate,scprep,magic,meld
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random,datetime
from sklearn import metrics
from sklearn import model_selection
from scipy import sparse
from scipy.stats import mannwhitneyu, tiecorrect, rankdata
from statsmodels.stats.multitest import multipletests
import scanpy as sc
import scvelo as scv
from adjustText import adjust_text
import warnings
from matplotlib import gridspec



# settings
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=9)
plt.rcParams['savefig.dpi']=600
sc.settings.verbosity=2
sc._settings.ScanpyConfig.n_jobs=-1
sns.set_style("ticks")


In [2]:

# fps
sdfp = '/home/ngr4/project/sccovid/shared/data/processed/'
pfp = '/home/ngr4/project/sccovid/results/'
pdfp = '/home/ngr4/project/sccovid/data/processed/'
sc.settings.figdir = pfp

def loader(fname,fpath,backed=None) : 
    start = time.time()
    adata = sc.read_h5ad(filename=os.path.join(fpath,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    return adata

def writer(fname,fpath,AnnData) :
    start = time.time()
    AnnData.write(os.path.join(fpath,fname))
    print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to save data'.format(time.time()-start))
    

if True :
    # load personal
    fname='scv2_200428.h5ad'
    %memit adata = loader(fname,pdfp)

    
# aesthetics
cmap_condition = {v:sns.cubehelix_palette(4,start=.5,rot=-.75)[i] for i,v in enumerate(adata.obs['Condition'].unique())}
cmap_infected = {'Infected':sns.color_palette('muted')[1],
                 'Bystander':sns.color_palette('muted')[0]}
cmap_ctype = {v:sns.color_palette('colorblind')[i] for i,v in enumerate(adata.obs['ctype'].unique())}

loaded @200504.11:30:55
took 42.50-s to load data
peak memory: 36591.26 MiB, increment: 36391.80 MiB


In [3]:
def mwu(X,Y,gene_names,correction=None,debug=False) :
    '''
    Benjamini-Hochberg correction implemented. Can change to Bonferonni

    gene_names (list)
    if X,Y single gene expression array, input x.reshape(-1,1), y.reshape(-1,1)

    NOTE: get zeros sometimes because difference (p-value is so small)
    '''
    p=pd.DataFrame()
    print('starting Mann-Whitney U w/Benjamini/Hochberg correction...\n')
    start = time.time()
    for i,g in enumerate(gene_names) :
        if i==np.round(np.quantile(np.arange(len(gene_names)),0.25)) :
            print('... 25% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.5)) :
            print('... 50% completed in {:.2f}-s'.format(time.time()-start))
        elif i==np.round(np.quantile(np.arange(len(gene_names)),0.75)) :
            print('... 75% completed in {:.2f}-s'.format(time.time()-start))
        p.loc[i,'Gene']=g
        if (tiecorrect(rankdata(np.concatenate((np.asarray(X[:,i]),np.asarray(Y[:,i])))))==0) :
            if debug :
                print('P-value not calculable for {}'.format(g))
            p.loc[i,'pval']=np.nan
        else :
            _,p.loc[i,'pval']=mannwhitneyu(X[:,i],Y[:,i]) # continuity correction is True
    print('\n... mwu computed in {:.2f}-s\n'.format(time.time() - start))
    if True :
        # ignore NaNs, since can't do a comparison on these (change numbers for correction)
        p_corrected = p.loc[p['pval'].notna(),:]
        if p['pval'].isna().any():
            print('Following genes had NA p-val:')
            for gene in p['Gene'][p['pval'].isna()]:
                print('  %s' % gene)
    else : 
        p_corrected = p
    new_pvals = multipletests(p_corrected['pval'],method='fdr_bh')
    p_corrected['pval_corrected'] = new_pvals[1]
    return p_corrected

def log2aveFC(X,Y,gene_names,AnnData=None) :
    '''not sensitivity to directionality due to subtraction

    X and Y full arrays, subsetting performed here

    `gene_names` (list): reduced list of genes to calc

    `adata` (sc.AnnData): to calculate reduced list. NOTE: assumes X,Y drawn from adata.var_names
    '''
    if not AnnData is None :
        g_idx = [i for i,g in enumerate(AnnData.var_names) if g in gene_names]
        fc=pd.DataFrame({'Gene':AnnData.var_names[g_idx],
                         'log2FC':np.log2(X[:,g_idx].mean(axis=0)) - np.log2(Y[:,g_idx].mean(axis=0))}) # returns NaN if negative value 
    else :
        fc=pd.DataFrame({'Gene':gene_names,
                         'log2FC':np.log2(X.mean(axis=0)) - np.log2(Y.mean(axis=0))})
    return fc

# Compare infected/bysander at each time point

## in all cells

In [6]:
fname = '123dpi_infVbystander'

dge = pd.DataFrame()
for t in ['123dpi']:#adata.obs['timepoint'].unique():
    print('\nstarting timepoint {}\n'.format(t))
    start_t=time.time()
    tdata = sc.AnnData(adata[(adata.obs['Condition']=='1dpi')|(adata.obs['Condition']=='2dpi')|(adata.obs['Condition']=='3dpi'),:])
#     tdata=tdata[:,~tdata.var_names.isin(['Malat1'])] # exclude outlier genes
    
    cluster='ctype'
    groups = tdata.obs[cluster].unique()
    for i in groups :
        start = time.time()
        
        print('  group: {}\n'.format(i))
        
        X = tdata.X[np.where((tdata.obs[cluster]==i) & (tdata.obs['scv2_10+']==0))[0],:].todense()
        X_mut = tdata.X[np.where((tdata.obs[cluster]==i) & (tdata.obs['scv2_10+']==1))[0],:].todense()
            
        X = np.asarray(X)
        X_mut = np.asarray(X_mut)
        

        p = mwu(X,X_mut,tdata.var_names) # directionality doesn't matter
        emd = scprep.stats.differential_expression(X_mut,X,
                                                   measure = 'emd',
                                                   direction='both', 
                                                   gene_names=tdata.var_names,
                                                   n_jobs=-1)
        emd['Gene']=emd.index
        emd=emd.drop(columns='rank')
        fc = log2aveFC(X_mut,X,tdata.var_names.to_list())
        gene_mismatch = fc['Gene'].isin(p['Gene'])
        if gene_mismatch.any():
            fc = fc.loc[gene_mismatch,:]
            warnings.warn('Warning: {} genes dropped due to p-val NA.'.format((gene_mismatch==False).sum()))
        dt = pd.merge(p,fc,how='left',on="Gene")
        gene_mismatch = emd['Gene'].isin(p['Gene'])
        if gene_mismatch.any():
            emd = emd.loc[gene_mismatch,:]
        dt = pd.merge(dt,emd,how='left',on='Gene')
        dt['Cell type']=[i]*dt.shape[0]
        dt['timepoint']=[t]*dt.shape[0]
        dt['nlog10pvalcorrected']=(-1)*np.log10(dt['pval_corrected'])
        
        if False:
            
            # PLOT?

            ## aesthetics
            cmap_volcano={'N.S.':'#E8E8E866','Up':'#d9534f','Down':'#428bca','B.H. cutoff':'#E8E8E8'}
            bonferonni_fulldata=0.01/3e5 # pval threshold
            dt['Significance']=['N.S.']*dt.shape[0]
            dt['Significance'][(dt['pval_corrected']<=0.01)]='B.H. cutoff'
            #     dt['Significance'][(dt['log2FC']>=1) & (dt['pval_corrected']<=bonferonni_fulldata)]='up'
            #     dt['Significance'][(dt['log2FC']<=-1) & (dt['pval_corrected']<=bonferonni_fulldata)]='down'
            dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']>0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Up'
            dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']<0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Down'

            ## plot
            fig,ax=plt.subplots(1,3,figsize=(12,3))
            p0=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                              data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                               ax=ax[0],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            p0top=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                              data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],ax=ax[0],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            ax[0].plot([dt['log2FC'].min(),dt['log2FC'].max()],
                    (-1)*np.log10([0.01,0.01]),
                    'k--',lw=0.5) # BH cut off
            ax[0].set(xlabel='$\log_2 \dfrac{Infected}{Bystander}$',ylabel='$-\log_{10}$($p_{corrected}$)')

            p1=sns.scatterplot(x='log2FC',y='emd',
                              data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                               ax=ax[1],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            p1top=sns.scatterplot(x='log2FC',y='emd',
                              data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                               ax=ax[1],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            ax[1].set(xlabel='$\log_2 \dfrac{Infected}{WT}$',ylabel='EMD(Infected,Bystander)')

            p2=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                              data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                               ax=ax[2],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            p2top=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                              data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                               ax=ax[2],s=3,edgecolor='none',
                              palette=cmap_volcano,hue='Significance',legend=False,
                              rasterized=True)
            ax[2].plot([dt['emd'].min(),dt['emd'].max()],
                    (-1)*np.log10([0.01,0.01]),
                    'k--',lw=0.5) # BH cut off
            ax[2].set(xlabel='EMD(Infected,Bystander)',ylabel='$-\log_{10}$($p_{corrected}$)')

            fig.tight_layout()

            ## annotate
            if False :
                ### pick genes w/mutual highest rank
                idx_up = (dt['nlog10pvalcorrected'][dt['Significance']=='up'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='up'].rank(ascending=False)).sort_values().index[0:10]
                g_up = tdata.var_names[idx_up]
                g_up_x = dt['log2FC'][idx_up].to_list()
                g_up_y = dt['nlog10pvalcorrected'][idx_up].to_list()
                idx_down = (dt['nlog10pvalcorrected'][dt['Significance']=='down'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='down'].rank(ascending=True)).sort_values().index[0:10]
                g_down = tdata.var_names[idx_down]
                g_down_x = dt['log2FC'][idx_down].to_list()
                g_down_y = dt['nlog10pvalcorrected'][idx_down].to_list()

                texts = [ax.text(g_up_x[i], g_up_y[i], g,fontsize=4) for i,g in enumerate(g_up)]
                adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
                texts_down = [ax.text(g_down_x[i], g_down_y[i], g,fontsize=4) for i,g in enumerate(g_down)]
                adjust_text(texts_down,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
            else : 
                ### manually pick 
                idx_g={}
                for g in genes_volcano[fname]: # match fname with dict key
                    idx_g[g]=[i for i,v in enumerate(dt['Gene']) if v==g][0] # assumes unique namees

                texts = [ax[0].text(dt['log2FC'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[0])

                texts = [ax[1].text(dt['log2FC'][i],dt['emd'][i],g,fontsize=6) for g,i in idx_g.items()]
                adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[1])

                texts = [ax[2].text(dt['emd'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[2])

            if True :
                # save plot
                fig.savefig(os.path.join(pfp,'volcano_{}_t{}.pdf'.format(fname,t)),dpi=300, bbox_inches='tight')
            
        dge = dge.append(dt, ignore_index=True)
        print('... computed in {:.2f}-s'.format(time.time()-start))
    print('\nFinished timepoint {} in {:.2f}-min'.format(t,(time.time()-start_t)/60))
    del tdata
if True :
    # save volcano plot data
    dge.to_csv(os.path.join(pfp,'dge_'+fname+'.csv'),index=False)



starting timepoint 123dpi

  group: Ciliated cells

starting Mann-Whitney U w/Benjamini/Hochberg correction...

... 25% completed in 36.58-s
... 50% completed in 76.71-s
... 75% completed in 121.83-s

... mwu computed in 171.28-s

Following genes had NA p-val:
  AL357873.1
  AL359815.1
  AL445648.1
  AL603839.1
  AC119428.2
  AL583808.2
  AC093580.1
  DNASE2B
  CLCA1
  AC093117.1
  AC095033.1
  LINC01709
  GSTM5
  KCNA2
  AL445426.1
  CD2
  LCE5A
  KPRP
  SMCP
  NTRK1
  SPTA1
  KCNJ9
  FCGR2B
  Z99758.1
  AL359853.2
  LINC00303
  ERLNC1
  AL445493.2
  LINC01352
  CCDC185
  AL353593.2
  AL137793.1
  LINC01737
  AL391832.1
  AL359924.1
  AL365184.1
  AC092687.2
  MYCN
  AC010096.1
  RMDN2-AS1
  AC009229.2
  LINC01820
  AC016722.1
  GTF2A1L
  LHCGR
  AC018462.1
  AC007422.2
  EMX1
  CD8B
  AC233266.2
  IGKV1OR2-108
  DPP10
  AC079988.1
  RAB6D
  AC073869.5
  AC068492.1
  THSD7B
  AC005042.2
  KCNH7
  AC016723.1
  HOXD8
  AC096667.1
  HSPE1-MOB4
  AC007163.1
  CTLA4
  LINC01280
  AC114803

In [5]:
# filter the dge a bit
if False:
    # load
    dge = pd.read_csv(os.path.join(pfp,'volcano_'+fname+'.csv'))
dge.replace([np.inf, -np.inf], np.nan, inplace=True)
dge.dropna(subset=['nlog10pvalcorrected','log2FC'], inplace=True)
dge = dge.loc[dge['emd'] < 10, :]

# Compare across time vs. MOCK

## in ciliated cells

In [7]:
# 3dpi_ciliated 
fname = '123dpi_infVmock'

dge = pd.DataFrame()
mock=sc.AnnData(adata[(adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==0) & (adata.obs[cluster]==i),:]) # force not view
for t in ['123dpi']:#adata.obs['timepoint'].unique():
    print('\nstarting timepoint {}\n'.format(t))
    start_t=time.time()
    tdata = sc.AnnData(adata[(adata.obs['Condition']=='1dpi')|(adata.obs['Condition']=='2dpi')|(adata.obs['Condition']=='3dpi'),:])
#     tdata=tdata[:,~tdata.var_names.isin(['Malat1'])] # exclude outlier genes
    
    cluster='ctype'
    groups = tdata.obs[cluster].unique() 
    for i in groups :
        start = time.time()
        
        print('  group: {}\n'.format(i))

        
        X = mock.X.todense()
        X_mut = tdata.X[np.where((tdata.obs[cluster]==i) & (tdata.obs['scv2_10+']==1))[0],:].todense()
            
        X = np.asarray(X)
        X_mut = np.asarray(X_mut)
        
        if X.shape[0]==0 or X_mut.shape[0]==0:
            print('  Empty group detectected for\n  t={}, cluster={}'.format(t,cluster))
            print('  ... skipping.')
            continue
        else:
        

            p = mwu(X,X_mut,tdata.var_names) # directionality doesn't matter
            emd = scprep.stats.differential_expression(X_mut,X,
                                                       measure = 'emd',
                                                       direction='both', 
                                                       gene_names=tdata.var_names,
                                                       n_jobs=-1)
            emd['Gene']=emd.index
            emd=emd.drop(columns='rank')
            fc = log2aveFC(X_mut,X,tdata.var_names.to_list())
            gene_mismatch = fc['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                fc = fc.loc[gene_mismatch,:]
                warnings.warn('Warning: {} genes dropped due to p-val NA.'.format((gene_mismatch==False).sum()))
            dt = pd.merge(p,fc,how='left',on="Gene")
            gene_mismatch = emd['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                emd = emd.loc[gene_mismatch,:]
            dt = pd.merge(dt,emd,how='left',on='Gene')
            dt['Cell type']=[i]*dt.shape[0]
            dt['timepoint']=[t]*dt.shape[0]
            dt['nlog10pvalcorrected']=(-1)*np.log10(dt['pval_corrected'])

            if False:

                # PLOT?

                ## aesthetics
                cmap_volcano={'N.S.':'#E8E8E866','Up':'#d9534f','Down':'#428bca','B.H. cutoff':'#E8E8E8'}
                bonferonni_fulldata=0.01/3e5 # pval threshold
                dt['Significance']=['N.S.']*dt.shape[0]
                dt['Significance'][(dt['pval_corrected']<=0.01)]='B.H. cutoff'
                #     dt['Significance'][(dt['log2FC']>=1) & (dt['pval_corrected']<=bonferonni_fulldata)]='up'
                #     dt['Significance'][(dt['log2FC']<=-1) & (dt['pval_corrected']<=bonferonni_fulldata)]='down'
                dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']>0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Up'
                dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']<0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Down'

                ## plot
                fig,ax=plt.subplots(1,3,figsize=(12,3))
                p0=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[0],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p0top=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],ax=ax[0],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[0].plot([dt['log2FC'].min(),dt['log2FC'].max()],
                        (-1)*np.log10([0.01,0.01]),
                        'k--',lw=0.5) # BH cut off
                ax[0].set(xlabel='$\log_2 \dfrac{Infected}{Bystander}$',ylabel='$-\log_{10}$($p_{corrected}$)')

                p1=sns.scatterplot(x='log2FC',y='emd',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[1],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p1top=sns.scatterplot(x='log2FC',y='emd',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                                   ax=ax[1],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[1].set(xlabel='$\log_2 \dfrac{Infected}{WT}$',ylabel='EMD(Infected,Bystander)')

                p2=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[2],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p2top=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                                   ax=ax[2],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[2].plot([dt['emd'].min(),dt['emd'].max()],
                        (-1)*np.log10([0.01,0.01]),
                        'k--',lw=0.5) # BH cut off
                ax[2].set(xlabel='EMD(Infected,Bystander)',ylabel='$-\log_{10}$($p_{corrected}$)')

                fig.tight_layout()

                ## annotate
                if False :
                    ### pick genes w/mutual highest rank
                    idx_up = (dt['nlog10pvalcorrected'][dt['Significance']=='up'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='up'].rank(ascending=False)).sort_values().index[0:10]
                    g_up = tdata.var_names[idx_up]
                    g_up_x = dt['log2FC'][idx_up].to_list()
                    g_up_y = dt['nlog10pvalcorrected'][idx_up].to_list()
                    idx_down = (dt['nlog10pvalcorrected'][dt['Significance']=='down'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='down'].rank(ascending=True)).sort_values().index[0:10]
                    g_down = tdata.var_names[idx_down]
                    g_down_x = dt['log2FC'][idx_down].to_list()
                    g_down_y = dt['nlog10pvalcorrected'][idx_down].to_list()

                    texts = [ax.text(g_up_x[i], g_up_y[i], g,fontsize=4) for i,g in enumerate(g_up)]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
                    texts_down = [ax.text(g_down_x[i], g_down_y[i], g,fontsize=4) for i,g in enumerate(g_down)]
                    adjust_text(texts_down,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
                else : 
                    ### manually pick 
                    idx_g={}
                    for g in genes_volcano[fname]: # match fname with dict key
                        idx_g[g]=[i for i,v in enumerate(dt['Gene']) if v==g][0] # assumes unique namees

                    texts = [ax[0].text(dt['log2FC'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[0])

                    texts = [ax[1].text(dt['log2FC'][i],dt['emd'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[1])

                    texts = [ax[2].text(dt['emd'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[2])

                if True :
                    # save plot
                    fig.savefig(os.path.join(pfp,'volcano_{}_t{}.pdf'.format(fname,t)),dpi=300, bbox_inches='tight')

            dge = dge.append(dt, ignore_index=True)
            print('... computed in {:.2f}-s'.format(time.time()-start))
    print('\nFinished timepoint {} in {:.2f}-min'.format(t,(time.time()-start_t)/60))
    del tdata
if True :
    # save volcano plot data
    dge.to_csv(os.path.join(pfp,'dge_'+fname+'.csv'),index=False)



starting timepoint 123dpi

  group: Ciliated cells

starting Mann-Whitney U w/Benjamini/Hochberg correction...

... 25% completed in 19.82-s
... 50% completed in 43.70-s
... 75% completed in 73.10-s

... mwu computed in 107.66-s

Following genes had NA p-val:
  AL627309.3
  AL732372.1
  AL669831.2
  AL645608.5
  AL139246.1
  LINC00982
  LINC01346
  CHD5
  AL359881.2
  AL359881.1
  Z98884.1
  AL357552.2
  C1orf127
  MASP2
  ANGPTL7
  AL031731.1
  NPPB
  AL357873.1
  C1orf195
  AL031283.3
  AL121992.2
  ARHGEF19-AS1
  IGSF21
  PAX7
  AL031727.2
  AL391883.1
  PLA2G2D
  PLA2G2F
  VWA5B1
  AL031728.1
  AL359815.1
  CELA3B
  CNR2
  AL591178.1
  MYOM3
  AL590683.1
  AL138902.1
  AL445648.1
  AL050344.1
  C1orf232
  FO393419.2
  AL445490.1
  AL512288.1
  AL353622.2
  AL357500.1
  NKAIN1
  AC114488.1
  AL513327.3
  AL354864.1
  EPHA10
  AL929472.2
  AL442071.1
  HPCAL4
  AL033527.3
  AL603839.1
  AL603839.2
  AC098484.3
  AC099795.1
  TIE1
  AL451062.3
  AL358075.1
  CYP4A11
  AC099788.1
  AL

In [8]:
# filter the dge a bit
if False:
    # load
    dge = pd.read_csv(os.path.join(pfp,'volcano_'+fname+'.csv'))
dge.replace([np.inf, -np.inf], np.nan, inplace=True)
dge.dropna(subset=['nlog10pvalcorrected','log2FC'], inplace=True)
dge = dge.loc[dge['emd'] < 10, :]

## Bystander over time



In [8]:
# 3dpi_ciliated 
fname = '123dpi_bystanderVmock'

dge = pd.DataFrame()
mock=sc.AnnData(adata[(adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==0) & (adata.obs[cluster]==i),:]) # force not view
for t in ['123dpi']:#adata.obs['timepoint'].unique():
    print('\nstarting timepoint {}\n'.format(t))
    start_t=time.time()
    tdata = sc.AnnData(adata[(adata.obs['Condition']=='1dpi')|(adata.obs['Condition']=='2dpi')|(adata.obs['Condition']=='3dpi'),:])
#     tdata=tdata[:,~tdata.var_names.isin(['Malat1'])] # exclude outlier genes
    
    cluster='ctype'
    groups = tdata.obs[cluster].unique() 
    for i in groups :
        start = time.time()

        print('  group: {}\n'.format(i))
        
        X = mock.X.todense()
        X_mut = tdata.X[np.where((tdata.obs[cluster]==i) & (tdata.obs['scv2_10+']==0))[0],:].todense()
            
        X = np.asarray(X)
        X_mut = np.asarray(X_mut)
        
        if X.shape[0]==0 or X_mut.shape[0]==0:
            print('  Empty group detectected for\n  t={}, cluster={}'.format(t,cluster))
            print('  ... skipping.')
            continue
        else:
        

            p = mwu(X,X_mut,tdata.var_names) # directionality doesn't matter
            emd = scprep.stats.differential_expression(X_mut,X,
                                                       measure = 'emd',
                                                       direction='both', 
                                                       gene_names=tdata.var_names,
                                                       n_jobs=-1)
            emd['Gene']=emd.index
            emd=emd.drop(columns='rank')
            fc = log2aveFC(X_mut,X,tdata.var_names.to_list())
            gene_mismatch = fc['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                fc = fc.loc[gene_mismatch,:]
                warnings.warn('Warning: {} genes dropped due to p-val NA.'.format((gene_mismatch==False).sum()))
            dt = pd.merge(p,fc,how='left',on="Gene")
            gene_mismatch = emd['Gene'].isin(p['Gene'])
            if gene_mismatch.any():
                emd = emd.loc[gene_mismatch,:]
            dt = pd.merge(dt,emd,how='left',on='Gene')
            dt['Cell type']=[i]*dt.shape[0]
            dt['timepoint']=[t]*dt.shape[0]
            dt['nlog10pvalcorrected']=(-1)*np.log10(dt['pval_corrected'])

            if False:

                # PLOT?

                ## aesthetics
                cmap_volcano={'N.S.':'#E8E8E866','Up':'#d9534f','Down':'#428bca','B.H. cutoff':'#E8E8E8'}
                bonferonni_fulldata=0.01/3e5 # pval threshold
                dt['Significance']=['N.S.']*dt.shape[0]
                dt['Significance'][(dt['pval_corrected']<=0.01)]='B.H. cutoff'
                #     dt['Significance'][(dt['log2FC']>=1) & (dt['pval_corrected']<=bonferonni_fulldata)]='up'
                #     dt['Significance'][(dt['log2FC']<=-1) & (dt['pval_corrected']<=bonferonni_fulldata)]='down'
                dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']>0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Up'
                dt['Significance'][((dt['pval_corrected']<=0.01) & (dt['log2FC']<0) & ([True if i in genes_volcano[fname] else False for i in dt['Gene']]))]='Down'

                ## plot
                fig,ax=plt.subplots(1,3,figsize=(12,3))
                p0=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[0],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p0top=sns.scatterplot(x='log2FC',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],ax=ax[0],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[0].plot([dt['log2FC'].min(),dt['log2FC'].max()],
                        (-1)*np.log10([0.01,0.01]),
                        'k--',lw=0.5) # BH cut off
                ax[0].set(xlabel='$\log_2 \dfrac{Infected}{Bystander}$',ylabel='$-\log_{10}$($p_{corrected}$)')

                p1=sns.scatterplot(x='log2FC',y='emd',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[1],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p1top=sns.scatterplot(x='log2FC',y='emd',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                                   ax=ax[1],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[1].set(xlabel='$\log_2 \dfrac{Infected}{WT}$',ylabel='EMD(Infected,Bystander)')

                p2=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']!='Up') | (dt['Significance']!='Down'))],
                                   ax=ax[2],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                p2top=sns.scatterplot(x='emd',y='nlog10pvalcorrected',
                                  data=dt.loc[((dt['Significance']=='Up') | (dt['Significance']=='Down'))],
                                   ax=ax[2],s=3,edgecolor='none',
                                  palette=cmap_volcano,hue='Significance',legend=False,
                                  rasterized=True)
                ax[2].plot([dt['emd'].min(),dt['emd'].max()],
                        (-1)*np.log10([0.01,0.01]),
                        'k--',lw=0.5) # BH cut off
                ax[2].set(xlabel='EMD(Infected,Bystander)',ylabel='$-\log_{10}$($p_{corrected}$)')

                fig.tight_layout()

                ## annotate
                if False :
                    ### pick genes w/mutual highest rank
                    idx_up = (dt['nlog10pvalcorrected'][dt['Significance']=='up'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='up'].rank(ascending=False)).sort_values().index[0:10]
                    g_up = tdata.var_names[idx_up]
                    g_up_x = dt['log2FC'][idx_up].to_list()
                    g_up_y = dt['nlog10pvalcorrected'][idx_up].to_list()
                    idx_down = (dt['nlog10pvalcorrected'][dt['Significance']=='down'].rank(ascending=False)+dt['log2FC'][dt['Significance']=='down'].rank(ascending=True)).sort_values().index[0:10]
                    g_down = tdata.var_names[idx_down]
                    g_down_x = dt['log2FC'][idx_down].to_list()
                    g_down_y = dt['nlog10pvalcorrected'][idx_down].to_list()

                    texts = [ax.text(g_up_x[i], g_up_y[i], g,fontsize=4) for i,g in enumerate(g_up)]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
                    texts_down = [ax.text(g_down_x[i], g_down_y[i], g,fontsize=4) for i,g in enumerate(g_down)]
                    adjust_text(texts_down,arrowprops=dict(arrowstyle="-", color='k', lw=0.25))
                else : 
                    ### manually pick 
                    idx_g={}
                    for g in genes_volcano[fname]: # match fname with dict key
                        idx_g[g]=[i for i,v in enumerate(dt['Gene']) if v==g][0] # assumes unique namees

                    texts = [ax[0].text(dt['log2FC'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[0])

                    texts = [ax[1].text(dt['log2FC'][i],dt['emd'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[1])

                    texts = [ax[2].text(dt['emd'][i],dt['nlog10pvalcorrected'][i],g,fontsize=6) for g,i in idx_g.items()]
                    adjust_text(texts,arrowprops=dict(arrowstyle="-", color='k', lw=0.5),ax=ax[2])

                if True :
                    # save plot
                    fig.savefig(os.path.join(pfp,'volcano_{}_t{}.pdf'.format(fname,t)),dpi=300, bbox_inches='tight')

            dge = dge.append(dt, ignore_index=True)
            print('... computed in {:.2f}-s'.format(time.time()-start))
    print('\nFinished timepoint {} in {:.2f}-min'.format(t,(time.time()-start_t)/60))
    del tdata
if True :
    # save volcano plot data
    dge.to_csv(os.path.join(pfp,'dge_'+fname+'.csv'),index=False)



starting timepoint 123dpi

  group: Ciliated cells

starting Mann-Whitney U w/Benjamini/Hochberg correction...

... 25% completed in 33.23-s
... 50% completed in 70.50-s
... 75% completed in 113.57-s

... mwu computed in 161.30-s

Following genes had NA p-val:
  AL627309.4
  AL645608.7
  UTS2
  AL357873.1
  AC004824.1
  AL359815.1
  AL445648.1
  AL603839.1
  AC119428.2
  AL583808.2
  AC093580.1
  DNASE2B
  CLCA1
  AC093117.1
  AC095033.1
  LINC01709
  GSTM5
  KCNA2
  AL445426.1
  CD2
  LINC01731
  CRNN
  LCE5A
  KPRP
  SMCP
  AL590666.1
  NTRK1
  SPTA1
  KCNJ9
  AL354714.1
  FCGR2B
  Z99758.1
  AL359853.2
  MYBPH
  LINC00303
  ERLNC1
  AC093422.2
  AL445493.2
  LINC01352
  CCDC185
  AL353593.2
  AL670729.1
  AL137793.1
  LINC01737
  AL391832.1
  ACTN2
  AL359924.1
  AL365184.1
  AC092687.2
  MYCN
  AC010096.1
  RMDN2-AS1
  AC009229.2
  LINC01820
  AC016722.1
  GTF2A1L
  LHCGR
  AC018462.1
  AC007422.2
  EMX1
  AC233266.2
  AC021188.1
  IGKV1OR2-108
  DPP10
  AC079988.1
  GYPC
  RAB6D


In [11]:
# filter the dge a bit
if False:
    # load
    dge = pd.read_csv(os.path.join(pfp,'volcano_'+fname+'.csv'))
dge.replace([np.inf, -np.inf], np.nan, inplace=True)
dge.dropna(subset=['nlog10pvalcorrected','log2FC'], inplace=True)
dge = dge.loc[dge['emd'] < 10, :]

In [9]:
fname

'123dpi_bystanderVmock'